# Neutronics simulation with CAD geometry

This example creates a CAD geometry and then carries out a neutronics simulation with a heating cell tally.

This section makes a few components and places them in a reactor object. These will form the CAD model with which we will perform a neutronics simulation.

In [ ]:
import paramak

plasma = paramak.Plasma(
    minor_radius=150.,
    major_radius=450.,
    triangularity=0.55,
    elongation=2.,
    rotation_angle=180,
    name='plasma'
)

blanket = paramak.BlanketFP(
    plasma=plasma,
    thickness=50,
    stop_angle=90,
    start_angle=-90,
    offset_from_plasma=40,
    rotation_angle=180,
    name='blanket'
)

my_reactor = paramak.Reactor([plasma, blanket])
my_reactor.show()

This next code block exports the 3D reactor geometry as a DAGMC compatibile h5m file which can be used as a neutronics geometry. DAGMC is a neutronics code that allows particle transport on CAD geometry with various neutronics transport codes (including OpenMC).

More details on DAGMC here https://svalinn.github.io/DAGMC/

This export will take a reasonable amount of time compared to other cells

In [ ]:
!rm dagmc.h5m
my_reactor.export_dagmc_h5m(filename='dagmc.h5m', min_mesh_size=15, max_mesh_size=30)

The model can now be simulated in OpenMC. OpenMC is imported along with some convenience packages

In [ ]:
import openmc
import neutronics_material_maker as nmm  # makes materials from a database

This section forms the neutronics model by combining the DAGMC model with a plasma source and some assigned materials. Additionally, the tallies to record the heating are specified. The code block also sets simulation intensity and specifies the neutronics results to record (know as tallies).

In [ ]:
# makes a bounding box around the CAD geometry with vacuum surfaces
bound_dag_univ = openmc.DAGMCUniverse(filename='dagmc.h5m').bounded_universe()
my_geometry = openmc.Geometry(root=bound_dag_univ)


# this links the material tags in the dagmc h5m file with materials.
# the materials names are changed after their creation to match the 
# expected material tags in the dagmc file.
mat1 = nmm.Material.from_library(name='DT_plasma').openmc_material
mat1.name = 'plasma'
mat2 = nmm.Material.from_library(name='Li4SiO4').openmc_material
mat2.name = 'blanket'

materials = openmc.Materials([mat1, mat2])


tally1 = openmc.Tally()
material_filter = openmc.MaterialFilter(mat2)  # the center column material
tally1 = openmc.Tally(name='blanket_heating')
tally1.filters = [material_filter]
tally1.scores = ['heating']

my_tallies = openmc.Tallies([tally1])

# here an instance of the settings object has been created and attributes set on creation
my_settings = openmc.Settings(batches = 1, particles = 100, run_mode = 'fixed source')

# Assigns a ring source of DT energy neutrons to the source. This source has a
# 14MeV neutron energy, with a radius of 350cm and is half a ring (0 to 180 degrees)
# If you are keen to make more realistic plasma sources take a looks at the
# openmc_plasma_source python package https://github.com/fusion-energy/openmc-plasma-source
my_source = openmc.Source()
my_source.angle = openmc.stats.Isotropic()
my_source.energy = openmc.stats.Discrete([14e6], [1])
my_source.space = openmc.stats.CylindricalIndependent(
    r=openmc.stats.Discrete([350], [1]),  # all source points at a radius of 350 cm
    phi=openmc.stats.Uniform(a=0, b=3.14), # angular distribution between 0 and pi
    z=openmc.stats.Discrete([0], [1]), # all source points at 0 on the z axis
    origin=(0.0, 0.0, 0.0), # centered around 0,0,0 x,y,z
)
my_settings.source = my_source

my_model = openmc.Model(
    materials=materials, geometry=my_geometry, settings=my_settings, tallies=my_tallies
)

# delete old files just in case they are there
!rm summary.h5
!rm statepoint*.h5

# starts the simulation and gets the filename of the output file produced
statepoint_file = my_model.run()

Extracting the resulting tally value from the statepoint file

In [ ]:
sp = openmc.StatePoint(statepoint_file)

heating_tally = sp.get_tally(name='blanket_heating')

# this returns the tally in unknown base units
heating_tally.get_values().sum()

This next cell just imports a post processing package and uses that package to find the units of the tally

In [ ]:
import openmc_tally_unit_converter as otuc

tally = otuc.process_tally(
    heating_tally,
)
# this prints the tally with base units identified
print(tally)

Converts the units from the base units (eV/simulated particle) to MeV/simulated particle

In [ ]:
otuc.process_tally(
    heating_tally,
    required_units="MeV/source_particle"
)

Converts units by normalising by the source strength (number of nuetrons per second)

In [ ]:
otuc.process_tally(
    heating_tally,
    required_units="gigawatts",
    source_strength=1e21
)

**Learning Outcomes from Task 11**

- Neutronics simulations can be performed by combining CAD models, neutron sources and material definitions.
- Scaled the units of the output results using post processing package